In [1]:
from __future__ import print_function, division
import math
import copy
import turtle

# control polygon
cpList = [
    [(0,0), (100,200), (300,250), (400,0)],                         #0
    [(0,0), (100,-200), (300,250), (400,0)],                        #1
    [(0,0), (300,250), (400,-100), (200,0)],                        #2
    [(0,0), (400,300), (-200,350), (200,0)],                        #3
    [(0,0), (100,200), (300,250)],                                  #4
    [(0,0), (100,200), (300,250), (400,0)],                         #5
    [(0,0), (100,200), (300,250), (400,0), (300, -50)],             #6
    [(0,0), (100,200), (300,250), (400,0), (300, -50), (150, 50)]   #7
]  

cp = copy.copy(cpList[7])

for i in range(len(cp)): cp[i] = (cp[i][0] - 200, cp[i][1] - 100)

# draw line segments
def draw_segments(ttl, points, color):
    ttl.penup()
    ttl.goto(points[0])
    ttl.color(color)
    ttl.pendown()
    for (x, y) in points:
        ttl.goto(x, y)

def conbination(n, k):
    return math.factorial(n) / (math.factorial(k) * math.factorial(n - k))

# draw bezier curve using subdivision
def draw_bezier_subdiv(ttl, cp, color):
    # cp: control polygon
    t = 0.5 # t: parameter
    bcf = [] # forward Bezier curve
    bcb = [] # backward Bezier curve
    # set control polygon of forward Bezier curve
    pnt = copy.deepcopy(cp) # internally dividing points
    # calculate internally dividing points
    for k in range(len(cp)):
        bcf.append(pnt[0])
        bcb.append(pnt[len(cp) - k - 1])
        for i in range(len(cp) - k - 1):
            pnt[i] = (
                (1 - t) * pnt[i][0] + t * pnt[i + 1][0],
                (1 - t) * pnt[i][1] + t * pnt[i + 1][1]
            )
    
    # forward Bezier curve
    if flatness(bcf) == True:
        draw_segments(ttl, bcf, color)
    else:
        draw_bezier_subdiv(ttl, bcf, color)
    # fbackward Bezier curve
    if flatness(bcb) == True:
        draw_segments(ttl, bcb, color)
    else:
        draw_bezier_subdiv(ttl, bcb, color)
    #
    return()

# check the flatness
def flatness(cp):
    degree = len(cp) - 1
    if degree < 2:
        return(True)
    # baseline connecting endpoints
    a = cp[degree][1] - cp[0][1]
    b = -(cp[degree][0] - cp[0][0])
    c = -(a * cp[0][0] + b * cp[0][1])
    d = math.sqrt(a*a + b*b)
    # in case of short length (it's not exactly)
    if d < 2:
        return(True)
    # calculate the maximum distance
    max_dist = 0
    for i in range(degree - 1):
        # calculate the distance between baseline and point
        dist = abs(a * cp[i + 1][0] + b * cp[i + 1][1] + c) / d
        if dist > max_dist:
            max_dist = dist
    # check the flatness
    if max_dist < 2:
        return(True)
    else:
        return(False)

def bezier_subdiv(cp, t):
    # cp: control polygon
    # t: parameter
    pnt = copy.deepcopy(cp) # internally dividing points
    # calculate internally dividing points
    for k in range(1, len(cp)):
        for i in range(len(cp) - k):
            pnt[i] = (
                (1 - t) * pnt[i][0] + t * pnt[i + 1][0],
                (1 - t) * pnt[i][1] + t * pnt[i + 1][1]
            )
    return(pnt[0])
    
# calculate sequence of points on Bezier curve
def bezier(cp, ndot):
    # cp: controll polygon
    # ndot: number of dots
    bp = [] # dots on Bezier curve
    # calculate the coordinates of dots
    for i in range(ndot + 1):
        t = i / ndot
        l = len(cp)
        bp.append(bezier_subdiv(cp, t))
    return(bp)
    
# setup turtle
gamera = turtle.Turtle()
gamera.reset()
gamera.speed(0)
# draw control polygon
draw_segments(gamera, cp, "black")
# calculate dots on Bezier curve
bp = bezier(cp, 20)
# draw Bezier curve
# draw_segments(gamera, bp, "red")
# draw Bezier curve
draw_bezier_subdiv(gamera, cp, "red")
# wait for the user to close the window
turtle.mainloop()